In [ ]:
!pip install pandas pyarrow tqdm nltk sentence-transformers huggingface_hub pyyaml


In [ ]:
# Import necessary libraries
import os
import requests
from tqdm import tqdm
from xml.etree import ElementTree as ET
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from huggingface_hub import HfApi, create_repo
import yaml
import nltk

# Download NLTK data
nltk.download('punkt')


In [ ]:
# Read the PMIDs from the file
pmid_file_path = './pmids.txt'
with open(pmid_file_path, 'r') as f:
    pmids = f.read().splitlines()


In [ ]:
# Function to fetch a PubMed article in XML format by its PMID
def fetch_pubmed_article(pmid):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={pmid}&retmode=xml"
    response = requests.get(url)
    if response.status_code == 200:
        return response.content
    else:
        return None


In [ ]:
# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
# Fetch articles and process them
pmid_list = []
sentence_list = []
embedding_list = []

for pmid in tqdm(pmids, desc="Downloading PubMed Articles"):
    article_xml = fetch_pubmed_article(pmid)
    if article_xml:
        # Parse the XML to extract the abstract
        root = ET.fromstring(article_xml)
        abstract_texts = root.findall(".//AbstractText")
        try:
            abstract = " ".join(abstract_text.text for abstract_text in abstract_texts if abstract_text.text is not None)
        except TypeError:
            import warnings
            warnings.warn(f"PMID {pmid} has no abstract")
            continue
        
        if not abstract.strip():
            import warnings
            warnings.warn(f"PMID {pmid} has an empty abstract")
            continue

        # Tokenize the abstract into sentences
        sentences = sent_tokenize(abstract)
        
        if not sentences:
            import warnings
            warnings.warn(f"PMID {pmid} resulted in empty sentences after tokenization")
            continue

        # Generate embeddings for each sentence
        embeddings = model.encode(sentences)
        
        # Store the PMIDs, sentences, and embeddings
        pmid_list.extend([pmid] * len(sentences))
        sentence_list.extend(sentences)
        embedding_list.extend(embeddings)

# Create a DataFrame with PMIDs, sentences, and embeddings
data = {'PMID': pmid_list, 'sentence': sentence_list, 'embedding': embedding_list}
df = pd.DataFrame(data)

# Save the DataFrame to Parquet format
parquet_file_path = './data/sentence_embeddings.parquet'
df.to_parquet(parquet_file_path, engine='pyarrow', index=False)

print(f"Embeddings saved to {parquet_file_path}")

In [ ]:
# Generate metadata in venomx format
metadata = {
    'description': 'Embeddings of each sentence in the abstracts of about 6000 selected PubMed articles. Github repo with jupyter notebook and data files is here: https://github.com/justaddcoffee/embed_pubmed',
    'prefixes': {
        'PMID': 'https://pubmed.ncbi.nlm.nih.gov/PMID_'
    },
    'model': {
        'name': 'all-MiniLM-L6-v2'
    },
    'dataset': {
        'name': 'Pubmed July 2024',
        'url': 'https://huggingface.co/biomedical-translator/pubmed_sentence_embeddings'
    }
}

# Save the metadata to a YAML file
metadata_file_path = './data/metadata.yaml'
with open(metadata_file_path, 'w') as f:
    yaml.dump(metadata, f)

print(f"Metadata saved to {metadata_file_path}")

In [ ]:
# Upload to Hugging Face
repo_id = "biomedical-translator/pubmed_sentence_embeddings"
create_repo(repo_id, repo_type="dataset")

this_notebook_path = "pubmed_sentence_embeddings.ipynb"

api = HfApi()
files_to_upload = [parquet_file_path, metadata_file_path, this_notebook_path]

for file in files_to_upload:
    api.upload_file(
        path_or_fileobj=file,
        path_in_repo=file,
        repo_id=repo_id,
        repo_type="dataset"
    )

print(f"Files uploaded to Hugging Face in repository: {repo_id}")
